In [2]:
import pandas as pd
import numpy as np
import re
import sys
sys.path.append('/home/descombe/Desktop/photogrammetry-tools/src/utils.py')
from utils import get_sfm_data_as_dict

In [3]:
file = '/media/gargantua/1000-plane/0000-sfm/2021/0115/20210115-142818-20200225-00001-suisse-sion-0001-loop-11-normalized/geodesy/test_loop11.csv'
wkt = pd.read_csv(file, dtype=str)

In [4]:
wkt

,WKT,X,Y,Z,filename
0,POINT Z (2594426.0019446 1120060.68234485 494....,2594556.455321000423282,1120124.622873000102118,494.959768999999994,20200225-115646-795901.jpg
1,POINT Z (2594426.0020656 1120060.68231785 494....,2594556.455442000180483,1120124.622845999896526,494.959743000000003,20200225-115647-045904.jpg
2,POINT Z (2594426.0019836 1120060.68248085 494....,2594556.455360000021756,1120124.623008999973536,494.959841999999981,20200225-115647-295903.jpg
3,POINT Z (2594426.0020676 1120060.68231985 494....,2594556.455444000195712,1120124.622847999911755,494.959743000000003,20200225-115647-547785.jpg
4,POINT Z (2594426.0017456 1120060.68225985 494....,2594556.455122000072151,1120124.622787999920547,494.959630000000004,20200225-115647-795862.jpg
...,...,...,...,...,...
914,POINT Z (2594426.1239116 1120060.25450185 494....,2594556.577287999913096,1120124.195030000060797,494.960367000000019,20200225-120035-295900.jpg
915,POINT Z (2594426.1339596 1120060.16488385 494....,2594556.587335999589413,1120124.105412000091746,494.957365999999979,20200225-120035-545860.jpg
916,POINT Z (2594426.1381026 1120060.12578385 494....,2594556.591479000169784,1120124.066311999922618,494.956889999999987,20200225-120035-795898.jpg
917,POINT Z (2594426.1361066 1120060.13844485 494....,2594556.589482999872416,1120124.078972999937832,494.956045000000017,20200225-120036-045911.jpg


In [5]:
structure_dict, views_dict, extrinsics_dict, intrinsics = get_sfm_data_as_dict(file.replace('geodesy/test_loop11.csv','omvg/sfm_data_bin.json'))

In [7]:
extrinsics_dict

{0: {'rotation': [[0.039867138997931165,
    -0.04606244151337856,
    -0.9981427065855594],
   [0.01774516994904672, 0.9988119695649836, -0.04538456122072147],
   [0.9990474063684861, -0.015902859351215614, 0.04063716147636753]],
  'center': [1.93072063375214, 0.029042753152551268, -3.810313358152532]},
 1: {'rotation': [[0.03975225011017333,
    -0.04604729717614572,
    -0.9981479875419026],
   [0.017477103894087728, 0.9988168075667249, -0.04538210838746318],
   [0.9990567098271517, -0.01564069515679898, 0.04050998894542674]],
  'center': [1.9307427336505896, 0.029034096218696258, -3.810314328754704]},
 2: {'rotation': [[0.039418372221216996,
    -0.04563441931984521,
    -0.9981801899981637],
   [0.01721704750481943, 0.9988393037903368, -0.04498464714602054],
   [0.9990744542858887, -0.01541249418412612, 0.04015830943127336]],
  'center': [1.9307680624404775, 0.029053755085167898, -3.810415286633266]},
 3: {'rotation': [[0.04154449225612716,
    -0.04435824727659104,
    -0.9981514

In [7]:
len(extrinsics_dict) / 30

30.633333333333333

In [8]:
file_out = file.replace('test_loop11.csv','homologous.dat')
with open(file_out, 'w') as f:
    for row_id in range(0, len(wkt), 30):
        #filename = wkt.iloc[row_id]['filename']
        #filename_ = views_dict.get(row_id)['ptr_wrapper']['data']['filename']
        coor_before = extrinsics_dict.get(row_id)['center']
        coor_after = wkt.iloc[row_id]['WKT']
        coor_digits = re.sub("[^0-9]", " ", coor_after).strip()
        x_after = '.'.join(coor_digits.split()[0:2])
        y_after = '.'.join(coor_digits.split()[2:4])
        z_after = '.'.join(coor_digits.split()[4:])
        #x_after = wkt.iloc[row_id]['X.1']
        #y_after = wkt.iloc[row_id]['Y.1']
        #z_after = wkt.iloc[row_id]['Z']
        for coor in coor_before:
            f.write("%s " % coor)
        f.write("%s %s %s\n" % (x_after, y_after, z_after))


In [9]:
file_out

'/media/gargantua/1000-plane/0000-sfm/2021/0115/20210115-142818-20200225-00001-suisse-sion-0001-loop-11-normalized/geodesy/homologous.dat'

In [21]:
views_dict.get(row_id)['polymorphic_id']

1073741824

In [25]:
from glob import glob

In [30]:
glob('/media/scanvan/model/camera_40008603-40009302/2020*/*/dense_structure.xyz')

['/media/scanvan/model/camera_40008603-40009302/20200331/20200331-121055_20200220-155037-sion-loop-1-segment-dense/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200331/20200331-123532_20200220-155037-sion-loop-1-segment-dense/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200331/20200331-133954_20200220-155037-sion-loop-1-segment-dense/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200331/20200331-142206_20200220-155037-sion-loop-1-segment-dense/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200402/20200402-121322_20200220-155037-sion-loop-1-segment-test/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200402/20200402-162611_20200220-155037-sion-loop-1-segment-test/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009302/20200402/20200402-163858_20200220-155037-sion-loop-1-segment-test/dense_structure.xyz',
 '/media/scanvan/model/camera_40008603-40009

In [31]:
glob('/media/scanvan/model/camera_40008603-40009302/2020*/*/omvg')

['/media/scanvan/model/camera_40008603-40009302/20201002/20201002-043728_20200224-105844_sion-loop-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20201002/20201002-224038_20200224-110422_sion-loop-1-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20201013/20201013-144414_20200224-154421_sion-loop-7-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20200930/20200930-191922_20200218-114450_sion-test-loop-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20200930/20200930-195639_20200218-115257_sion-test-loop-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20200930/20200930-220958_20200218-115931_sion-test-loop-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20201001/20201001-014021_20200218-121135_sion-test-loop-2/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20201001/20201001-134620_20200220-155037_sion-loop-1/omvg',
 '/media/scanvan/model/camera_40008603-40009302/20201001/20201001-175336_20200220-155037_sion-loop-1-normalized/